### Individual users analysis

In [5]:
import itertools
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

import random
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
np.random.seed(0)

import pickle
import requests
from urllib.parse import urlencode
from sklearn.preprocessing import LabelEncoder
import json

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from plotnine import ggplot, aes, geom_bar, theme_minimal, labs

from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import accuracy_score, precision_score

from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz

import os

os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'  # Adjust the path as per your Graphviz installation

from sklearn.tree import _tree
from tqdm import tqdm

In [18]:
bs_token = 'lWYPGrEyZ0xqY7CJGTMx3DP60VBxZ21v7yrHjKguyGtQY2C5z16og6N8zu0R4Mbw'
host = 'td.winnerstudio.vip'

def pull_data(sql_script, bs_token, host):
    data = {
        'token':          bs_token,
        'format':         'json_object',
        'timeoutSeconds': 2000,
        'sql':            sql_script
    }
    data = str(urlencode(data))
    response = requests.post(f'http://{host}:8992/querySql?{data}', timeout = 1000000)
    # Sample list of JSON strings
    json_list = response.text.split('\n')[1:]
    # Convert JSON strings to dictionaries
    dict_list = []
    for json_str in json_list:
        try:
            dict_list.append(json.loads(json_str))
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
    # Create a pandas DataFrame
    data = pd.DataFrame(dict_list).sort_index(axis = 1)
    return data


In [7]:
def extract_within_hours(withdraw_times, first_pay_time, hours):
    # Convert withdraw_times to datetime
    withdraw_times_dt = pd.to_datetime(withdraw_times)
    
    # Convert first_pay_time to datetime (if it's not already)
    first_pay_time_dt = pd.to_datetime(first_pay_time)
    
    # Calculate the end time for comparison
    end_time = first_pay_time_dt + pd.Timedelta(hours=hours)
    
    # Return indexes where withdraw_times_dt is within the specified hours
    return [i for i, time in enumerate(withdraw_times_dt) if time <= end_time]

def extract_within_hours(withdraw_times, first_pay_time, hours):
    # Convert withdraw_times to datetime
    withdraw_times_dt = pd.to_datetime(withdraw_times)
    
    # Convert first_pay_time to datetime (if it's not already)
    first_pay_time_dt = pd.to_datetime(first_pay_time)
    
    # Calculate the end time for comparison
    end_time = first_pay_time_dt + pd.Timedelta(hours=hours)
    
    # Return indexes where withdraw_times_dt is within the specified hours
    return [i for i, time in enumerate(withdraw_times_dt) if time <= end_time]

def sum_withdraws_within_hours(withdraw_history, indexes):
    withdraw_history = str(withdraw_history).strip('[]')
    withdraw_values_list = [str(item.strip()) for item in withdraw_history.split(',')]
    
    # Ensure all values are numeric and calculate sum for selected indexes
    selected_withdraws = [float(withdraw_values_list[i].replace('[', '').replace(']', '')) for i in indexes 
                            if i < len(withdraw_values_list) 
                            and withdraw_values_list[i] is not None 
                            and withdraw_values_list[i].replace('[', '').replace(']', '') != ''
                            and withdraw_values_list[i].replace('[', '').replace(']', '') != '.']
    
    return sum(selected_withdraws)

def max_payments_in_window(payment_times, window_minutes):
    max_count = 0
    payment_times = pd.to_datetime(payment_times)
    for i, start_time in enumerate(payment_times):
        end_time = start_time + pd.Timedelta(minutes=window_minutes)
        count = np.sum((payment_times >= start_time) & (payment_times <= end_time))
        if count > max_count:
            max_count = count
    return max_count

def extract_second_payment(x):
    # Remove brackets and replace 'null' with 'NaN'
    cleaned = x.replace('[', '').replace(']', '').replace('null', 'NaN').split(',')
    # Check if there is a second element
    if len(cleaned) > 1:
        return float(cleaned[1])
    else:
        return np.nan
    
def extract_third_payment(x):
    # Remove brackets and replace 'null' with 'NaN'
    cleaned = x.replace('[', '').replace(']', '').replace('null', 'NaN').split(',')
    # Check if there is a second element
    if len(cleaned) > 1:
        return float(cleaned[2])
    else:
        return np.nan

def calculate_minutes_diff(start, end):
    if pd.notna(start) and pd.notna(end):
        return (end - start).total_seconds() / 60.0
    else:
        return np.nan

def get_unique_values(lst):
    return list(set(lst))

def extract_datetime(x, index):
    dates = x  # Convert JSON string to list
    if len(dates) > index:
        return pd.to_datetime(str(dates[index])[1:-1])
    else:
        return np.nan

# General function to sum payments within specified hours using the extracted indexes
def sum_payments_within_hours(payment_history, indexes):
    payment_history = str(payment_history).strip('[]')
    payment_values_list = [str(item.strip()) for item in payment_history.split(',')]
    selected_payments = [float(payment_values_list[i].replace('[', '').replace(']', '')) for i in indexes 
                            if i < len(payment_values_list) 
                            and payment_values_list[i] is not None 
                            and payment_values_list[i].replace('[', '').replace(']', '') != ''
                            and payment_values_list[i].replace('[', '').replace(']', '') != '.']
    
    return sum(selected_payments)

# Function to convert the string list to a list of datetime objects
def convert_to_datetime_list(string):
    string = str(string).strip('[]').replace("'", '"').replace('"', '').replace(" 2023", '2023').replace(" 2024", '2024').replace("     2023", '2023')

    string = string.strip()
    date_strs = string.split(',')
    date_strs = [date_str.strip() for date_str in date_strs]
    return pd.to_datetime(date_strs, format='%Y-%m-%d %H:%M:%S.%f')

# Define the model 
def create_model(criterion, depth, leaf_size):

    pipe = Pipeline([
        ('regressor', DecisionTreeClassifier(criterion = criterion, max_depth = depth, min_samples_leaf = leaf_size))
    ])

    return pipe

def traverse_tree(tree, node_id = 0, depth = 0, dataset = None, path = None):

    if dataset is None:
        dataset = pd.DataFrame(columns=["Event", "Disputer rate"])
    
    if path is None:
        path = []

    if tree.children_left[node_id] != _tree.TREE_LEAF:
    # if node_id != _tree.TREE_LEAF:
        feature_name = X_train.columns[tree.feature[node_id]]
        split_value = tree.threshold[node_id]
        
        path_left = path + [(f"{feature_name} ≤ {round(split_value, 3)}")]
        dataset = traverse_tree(tree, tree.children_left[node_id], depth + 1, dataset, path_left)
        
        path_right = path + [(f"{feature_name} > {round(split_value, 3)}")]
        dataset = traverse_tree(tree, tree.children_right[node_id], depth + 1, dataset, path_right) 
        # print('function here is okay')    
    
    else:
        leaf_size = np.sum(tree.value[node_id])
        # print('leaf_size', leaf_size)
        # print('else here is okay')
        # print(len(tree.value[node_id][0]))
        if len(tree.value[node_id][0]) > 1: 
            true_cases = tree.value[node_id][0][1]
        else: 
            true_cases = 0
        # print('else here is okay oh oh')
        # proportion_true = tree.value[node_id][0][1] / np.sum(tree.value[node_id])
        proportion_true = true_cases / leaf_size
        event = " & ".join([f"{event_name}" if " > " in event_name else f"{event_name}" for event_name in path])
        # print('有问题')
        dataset = pd.concat([dataset, pd.DataFrame({"Event": [event], "Disputer rate": [proportion_true],
                                                    "Payers": [leaf_size], "Disputers": [true_cases]})], ignore_index=True)

    return dataset


In [20]:
# So, you need to investigate how to increase the number of users across your platform. You need to understand the relevant parameters that will increase the performance of your game 
day = 7
version = 1

final_query = f"""

select  
    "#account_id"
    , "$part_event"
    , min("#event_time") as first_dispute 
    , max("#event_time") as last_dispute
    , count(distinct "#event_time") as total_disputes
from ta.v_event_59 
    where "$part_event" in ('pay_dispute', 'fraud') 
group by 1,2
order by 1,2

"""

print(final_query)



select  
    "#account_id"
    , "$part_event"
    , min("#event_time") as first_dispute 
    , max("#event_time") as last_dispute
    , count(distinct "#event_time") as total_disputes
from ta.v_event_59 
    where "$part_event" in ('pay_dispute', 'fraud') 
group by 1,2
order by 1,2




In [21]:
df = pull_data(final_query, bs_token, host)
df

""


In [ ]:

def analyze_user_behavior(df):
    user_analysis = []
    
    for _, row in df.iterrows():
        account_id = row['#account_id']
        sum_bet_7 = row['sum_bet_day_7']
        avg_bet_7 = row['avg_bet_day_7']
        max_bet_7 = row['max_bet_day_7']
        win_bet_7 = row['win_bet_day_7']
        
        behavior_notes = []

        # Check for high betting behavior
        if max_bet_7 > (avg_bet_7 * 10):
            behavior_notes.append(f"High single bet compared to average betting behavior (Max bet: {max_bet_7}, Avg bet: {avg_bet_7}).")

        # Check for consistent losses
        if win_bet_7 < 0:
            behavior_notes.append(f"User consistently losing in the last 7 days (Total loss: {win_bet_7}).")

        # Check for sudden changes in betting patterns
        if row['sum_bet_day_1'] != row['sum_bet_day_7']:
            behavior_notes.append(f"Betting pattern changed from Day 1 to Day 7 (Day 1 total bet: {row['sum_bet_day_1']}, Day 7 total bet: {sum_bet_7}).")

        # Check for high total bet amount that could lead to disputes
        if sum_bet_7 > 10000:  # arbitrary threshold for potential dispute risk
            behavior_notes.append(f"High total bet in the last 7 days: {sum_bet_7}. This might be a reason for a dispute.")
        
        if behavior_notes:
            user_analysis.append((account_id, behavior_notes))
    
    return user_analysis

# Perform the analysis on the dataset
user_behaviors = analyze_user_behavior(data)

# Display the first few user behavior analyses
user_behaviors[:5]
